In [22]:
import pandas as pd
import csv
import os
import pymongo
import json
from datetime import datetime, date
%load_ext blackcellmagic
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc, inspect, cast, Float,Integer

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [2]:
engine = create_engine("sqlite:///Data//covid19.sqlite")

In [3]:
inspector = inspect(engine)
table_names = inspector.get_table_names()

# schema = inspector.get_columns("icecreamstore")
# print(schema)
table_names

['global_covid_data', 'us_covid_data', 'us_lookup']

In [4]:
Base = automap_base()
Base.prepare(engine, reflect=True)
global_data = Base.classes.global_covid_data
us_data = Base.classes.us_covid_data
us_lookup = Base.classes.us_lookup
session = Session(engine)

In [ ]:
schema = inspector.get_columns("global_covid_data")
# schema

## All Country Global Data

In [ ]:
countries = session.query(global_data.Country_Region).distinct().order_by(global_data.Country_Region).all()

In [ ]:
global_covid_results = (
    session.query(
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered),
    )
    .group_by(global_data.Country_Region, global_data.Date)
    .order_by(global_data.Country_Region)
).all()

In [ ]:
def get_country_covid_data(country):
    data = {}
    for rec in global_covid_results:
        if rec[0] == country:
            data[rec[1]] = {"Confirmed_Cases" : int(rec[2]), "Deaths" : int(rec[3]), "Recovered" : int(rec[4])}
    return data

In [ ]:
global_dict  = [{  "Country": country[0],
     "Data" : get_country_covid_data(country[0])
  }for country in countries]
global_dict

In [31]:
# Not needed
# country_results = (
#     session.query(
#         global_data.ID,
#         global_data.Country_Region,
#         global_data.Date,
#         func.sum(global_data.Confirmed_Cases),
#         func.sum(global_data.Deaths),
#         func.sum(global_data.Recovered)
#     ).filter(global_data.Country_Region == country)
#      .group_by(global_data.ID, global_data.Country_Region, global_data.Date)
#     .order_by(global_data.Country_Region)
# ).all()

In [ ]:
# results = [{"Date": rec[1], data_point: int(rec[col_idx])} for rec in country_results]

In [21]:
# country list
qry = (
    session.query(global_data.Country_Region)
    .distinct()
    .order_by(global_data.Country_Region)
    .statement
)
df = pd.read_sql_query(qry, engine).rename(columns={"Country_Region": "Country"})
data = {"Country": df.Country.values.tolist()}
data

In [82]:
country = "Algeria"
qry =(
    session.query(
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered)
    ).filter(global_data.Country_Region == country)
     .group_by(global_data.Country_Region, global_data.Date)
    .order_by(global_data.Date)
).statement

In [83]:
df = pd.read_sql_query(qry, engine).rename(
    columns={
        "Country_Region": "Country",
        "sum_1": "Confirmed",
        "sum_2": "Deaths",
        "sum_3": "Recovered",
    })
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime('%Y-%m-%d')
df = df.sort_values(by=["Date"])
df["Deltas_Confirmed"] = df.Confirmed.diff()
df["Five_Day_Avg_Confirmed"] = df.Deltas_Confirmed.rolling(window = 5).mean()
df["Deltas_Deaths"] = df.Deaths.diff()
df["Five_Day_Avg_Deaths"] = df.Deltas_Deaths.rolling(window = 5).mean()
df["Deltas_Recovered"] = df.Recovered.diff()
df["Five_Day_Avg_Recovered"] = df.Deltas_Recovered.rolling(window = 5).mean()
df = df.fillna(0)

df.tail()

,Country,Date,Confirmed,Deaths,Recovered,Deltas_Confirmed,Five_Day_Avg_Confirmed,Deltas_Deaths,Five_Day_Avg_Deaths,Deltas_Recovered,Five_Day_Avg_Recovered
80,Algeria,2020-04-08,1572,205.0,237.0,104.0,80.2,12.0,20.0,124.0,35.0
81,Algeria,2020-04-09,1666,235.0,347.0,94.0,83.0,30.0,21.0,110.0,51.4
71,Algeria,2020-04-10,1761,256.0,405.0,95.0,88.2,21.0,20.8,58.0,63.0
72,Algeria,2020-04-11,1825,275.0,460.0,64.0,80.4,19.0,20.4,55.0,74.0
73,Algeria,2020-04-12,1914,293.0,591.0,89.0,89.2,18.0,20.0,131.0,95.6


In [39]:
data = {
    "Date": df.Date.values.tolist(),
    "Confirmed_Cases": df.Confirmed.values.tolist(),
    "Deltas_Confirmed": df.Deltas_Confirmed.values.tolist(),
    "Five_Day_Avg_Confirmed": df.Five_Day_Avg_Confirmed.values.tolist(),
    "Deaths": df.Deaths.values.tolist(),
    "Deltas_Deaths": df.Deltas_Deaths.values.tolist(),
    "Five_Day_Avg_Deaths": df.Five_Day_Avg_Deaths.values.tolist(),
    "Recovered": df.Recovered.values.tolist(),
    "Deltas_Recovered": df.Deltas_Recovered.values.tolist(),
    "Five_Day_Avg_Recovered": df.Five_Day_Avg_Recovered.values.tolist(),
    
}

In [133]:
# data

## Analysis

In [80]:

first_case_df = df.loc[df["Confirmed"] > 0 ]
first_case_df = first_case_df.head(1)
current_state_df = df.tail(2)
current_state_df["Confirmed_Status"] = current_state_df.Five_Day_Avg_Confirmed.diff()
current_state_df["Deaths_Status"] = current_state_df.Five_Day_Avg_Deaths.diff()
current_state_df["Recovered_Status"] = current_state_df.Five_Day_Avg_Recovered.diff()



first_case_df
# current_state_df

<ipython-input-80-5012d5a80e46>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-80-5012d5a80e46>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-80-5012d5a80e46>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Date,Confirmed,Deaths,Recovered,Deltas_Confirmed,Five_Day_Avg_Confirmed,Deltas_Deaths,Five_Day_Avg_Deaths,Deltas_Recovered,Five_Day_Avg_Recovered
0,China,2020-01-22,548,17.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
first_date = first_case_df.iloc[0, 1]
days_since_first_case = (
    (datetime.strptime(current_state_df.iloc[1, 1], "%Y-%m-%d"))
    - (datetime.strptime(first_date, "%Y-%m-%d"))
).days
confirmed_current = "{:,}".format(current_state_df.iloc[1, 2])
deaths_current = "{:,}".format(int(current_state_df.iloc[1, 3]))
recovered_current = "{:,}".format(int(current_state_df.iloc[1, 4]))
confirmed_status = int(current_state_df.iloc[1, 11])
deaths_status = (current_state_df.iloc[1, 12])
recovered_status = (current_state_df.iloc[1, 13])

summary = (
        f"The first case of COVID-19 in {country} was reported {days_since_first_case} days ago on "
        f"{datetime.strptime(first_date, '%Y-%m-%d').strftime('%B %d, %Y')}. "
        f"Since then, the country has reported {confirmed_current} cases, and {deaths_current} deaths. "
        f"The current number of recovered cases is {recovered_current}.")

In [105]:
days_since_first_case
confirmed_current
deaths_current
recovered_current
confirmed_status
deaths_status
recovered_status
summary


'The first case of COVID-19 in Algeria was reported 81 days ago on January 22, 2020. Since then, the country has reported 83,134 cases, and 3,343 deaths. The current number of recovered cases is 77,956.'

## GLobal Data Pull

In [106]:
qry = (
    session.query(
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered)
    )
     .group_by(global_data.Country_Region, global_data.Date)
    .order_by(global_data.Date)
).statement

In [108]:
df = pd.read_sql_query(qry, engine).rename(
    columns={
        "Country_Region": "Country",
        "sum_1": "Confirmed",
        "sum_2": "Deaths",
        "sum_3": "Recovered",
    }
).astype({"Confirmed": "int32", "Deaths": "int32","Recovered": "int32"})
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime('%Y-%m-%d')
df = df.sort_values(by = "Date")
# df.loc[df['Country'] == "China"].tail(20)

,Country,Date,Confirmed,Deaths,Recovered
10211,China,2020-03-24,81591,3281,73280
10396,China,2020-03-25,81661,3285,73773
10581,China,2020-03-26,81782,3291,74181
10766,China,2020-03-27,81897,3296,74720
10951,China,2020-03-28,81999,3299,75100
11136,China,2020-03-29,82122,3304,75582
11506,China,2020-03-30,82198,3308,75923
11691,China,2020-03-31,82279,3309,76206
12986,China,2020-04-01,82361,3316,76405
13726,China,2020-04-02,82432,3322,76565


In [112]:
data = {
    "Country": df.Country.values.tolist(),
    "Date": df.Date.values.tolist(),
    "Confirmed": df.Confirmed.values.tolist(),
    "Deaths": df.Deaths.values.tolist(),
    "Recovered": df.Recovered.values.tolist(),
}

In [113]:
data

{'Country': ['Afghanistan',
  'Namibia',
  'Nepal',
  'Netherlands',
  'New Zealand',
  'Nicaragua',
  'Niger',
  'Nigeria',
  'North Macedonia',
  'Norway',
  'Oman',
  'Pakistan',
  'Panama',
  'Papua New Guinea',
  'Paraguay',
  'Peru',
  'Philippines',
  'Poland',
  'Portugal',
  'Qatar',
  'Mozambique',
  'Romania',
  'Morocco',
  'Mongolia',
  'Latvia',
  'Lebanon',
  'Liberia',
  'Libya',
  'Liechtenstein',
  'Lithuania',
  'Luxembourg',
  'MS Zaandam',
  'Madagascar',
  'Malawi',
  'Malaysia',
  'Maldives',
  'Mali',
  'Malta',
  'Mauritania',
  'Mauritius',
  'Mexico',
  'Moldova',
  'Monaco',
  'Montenegro',
  'Rwanda',
  'Saint Kitts and Nevis',
  'Saint Lucia',
  'Timor-Leste',
  'Togo',
  'Trinidad and Tobago',
  'Tunisia',
  'Turkey',
  'US',
  'Uganda',
  'Ukraine',
  'United Arab Emirates',
  'United Kingdom',
  'Uruguay',
  'Uzbekistan',
  'Venezuela',
  'Vietnam',
  'West Bank and Gaza',
  'Western Sahara',
  'Yemen',
  'Zambia',
  'Zimbabwe',
  'Thailand',
  'Tanzani

## NICK S Plots

In [116]:
results = (
    session.query(
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered)
    )
     .group_by(global_data.Country_Region, global_data.Date)
    .order_by(global_data.Date)
).statement

In [117]:
df_cc = pd.read_sql_query(results, engine).rename(
    columns={
        "Country_Region": "Country",
        "sum_1": "Confirmed",
        "sum_2": "Deaths",
        "sum_3": "Recovered",
    }
).astype({'Confirmed': 'int32'})
df_cc["Date"] = pd.to_datetime(df_cc["Date"]).dt.strftime('%Y-%m-%d')
df_cc = df_cc.sort_values(by = "Date")
df_cc.loc[df_cc['Country'] == "China"].tail(10)

,Country,Date,Confirmed,Deaths,Recovered
13911,China,2020-04-03,82511,3326.0,76760.0
14096,China,2020-04-04,82543,3330.0,76946.0
14281,China,2020-04-05,82602,3333.0,77207.0
14466,China,2020-04-06,82665,3335.0,77310.0
14651,China,2020-04-07,82718,3335.0,77410.0
14836,China,2020-04-08,82809,3337.0,77567.0
15021,China,2020-04-09,82883,3339.0,77679.0
13171,China,2020-04-10,82941,3340.0,77791.0
13356,China,2020-04-11,83014,3343.0,77877.0
13541,China,2020-04-12,83134,3343.0,77956.0


In [118]:
df_countrydate = df_cc[df_cc['Confirmed']>0]
df_countrydate = df_countrydate.groupby(['Date','Country']).sum().reset_index()
df_countrydate = df_countrydate.sort_values(by = "Date")
df_countrydate

,Date,Country,Confirmed,Deaths,Recovered
0,2020-01-22,China,548,17.0,28.0
1,2020-01-22,Japan,2,0.0,0.0
2,2020-01-22,"Korea, South",1,0.0,0.0
3,2020-01-22,Taiwan*,1,0.0,0.0
4,2020-01-22,Thailand,2,0.0,0.0
...,...,...,...,...,...
7302,2020-04-12,Georgia,257,3.0,67.0
7303,2020-04-12,Germany,127854,3022.0,60300.0
7304,2020-04-12,Ghana,566,8.0,4.0
7295,2020-04-12,Eswatini,14,0.0,7.0


In [119]:
df_t = pd.read_csv("tourism1.csv")
df_t = df_t.drop(columns=['Unnamed: 0','Unnamed: 2' , 'Unnamed: 3', 'Unnamed: 4' , 'Unnamed: 5' , 'World Tourism Organization (2020), Compendium of Tourism Statistics dataset [Electronic], UNWTO, Madrid, data updated on 20/01/2020.'])
df_t = df_t.drop([0, 1, 2, 3, 4, 5])
df_t_clean = df_t.rename(columns={"Unnamed: 1": "Country", "Unnamed: 6": "Tourism"})
df_t_clean

df_merged = pd.merge(left=df_countries, right=df_t_clean, left_on='Country', right_on='Country')

FileNotFoundError: [Errno 2] File tourism1.csv does not exist: 'tourism1.csv'

In [121]:
df_countries = df_cc.groupby(['Country', 'Date']).max().reset_index().sort_values('Date', ascending=False)
df_countries = df_countries.drop_duplicates(subset = ['Country'])
df_countries = df_countries[df_countries['Confirmed']>0]

df_countries

,Country,Date,Confirmed,Deaths,Recovered
15169,Zimbabwe,2020-04-12,14,3.0,0.0
2459,Cabo Verde,2020-04-12,8,1.0,1.0
6067,Haiti,2020-04-12,33,3.0,0.0
8937,Malta,2020-04-12,378,3.0,44.0
2377,Burundi,2020-04-12,5,0.0,0.0
...,...,...,...,...,...
10577,Pakistan,2020-04-12,5230,91.0,1028.0
9511,Montenegro,2020-04-12,272,3.0,5.0
5575,Greece,2020-04-12,2114,98.0,269.0
11725,Saint Vincent and the Grenadines,2020-04-12,12,0.0,1.0


In [120]:
df_t = pd.read_csv("tourism1.csv")
df_t = df_t.drop(columns=['Unnamed: 0','Unnamed: 2' , 'Unnamed: 3', 'Unnamed: 4' , 'Unnamed: 5' , 'World Tourism Organization (2020), Compendium of Tourism Statistics dataset [Electronic], UNWTO, Madrid, data updated on 20/01/2020.'])
df_t = df_t.drop([0, 1, 2, 3, 4, 5])
df_t_clean = df_t.rename(columns={"Unnamed: 1": "Country", "Unnamed: 6": "Tourism"})
df_t_clean

df_merged = pd.merge(left=df_countries, right=df_t_clean, left_on='Country', right_on='Country')

FileNotFoundError: [Errno 2] File tourism1.csv does not exist: 'tourism1.csv'

## Virus Spread

In [122]:
fig3 = px.choropleth(df_countrydate, 
                    locations="Country", 
                    locationmode = "country names",
                    color="Confirmed", 
                    hover_name="Country", 
                    animation_frame="Date"
                   )

fig3.update_layout(
    title_text = 'Spread of Coronavirus',
    title_x = .6,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
fig3.write_image("virus_spread.png")   
fig3.show()

In [123]:
fig3.write_image("virus_spread.png")   

ValueError: Image generation requires the psutil package.

Install using pip:
    $ pip install psutil

Install using conda:
    $ conda install psutil


## Total Snapshot

In [75]:

line_data = df_cc.groupby('Date').sum().reset_index()

line_data = line_data.melt(id_vars='Date', 
                 value_vars=['Confirmed', 
                             'Recovered', 
                             'Deaths'], 
                 var_name='Ratio', 
                 value_name='Value')

fig_line = px.line(line_data, x="Date", y="Value", color='Ratio', 
              title='Confirmed cases, Recovered cases, and Death Over Time')
fig_line.show()

## Tourism vs Cases

In [104]:
fig = px.scatter(df_merged, x="Tourism", y="Confirmed" , labels={'Tourism':'Total amount of people visiting country in 2019', 'Confirmed':'Total amount of COVID19 Cases'},)
fig.show()